## PREPROCESSING

Remove inappropriate games
Remove rows that are not games
plots before PP
Removing NaN values
droping the columns
categorical to numerical
plots
Remove outliers
scaling
PCA
plots

---

CLUSTERING


In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from main_functions import *

In [ ]:
# import os

# print(os.getcwd())

In [6]:
# import tarfile

# Open the tar.gz file
# file_path = 'games.tar.gz'  # Adjust this if the path is different
# with tarfile.open(file_path, 'r:gz') as tar:
# tar.extractall('extracted_data')  # Extract all contents to 'extracted_data' directory

In [ ]:
# df = pd.read_csv('extracted_data/games.csv')
# df.head()

In [ ]:
df = pd.read_csv("games.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.head(5)

In [ ]:
columns_to_drop = [
    "AppID",
    "Full audio languages",
    "Header image",
    "Website",
    "Support url",
    "Support email",
    "Metacritic url",
    "Score rank",
    "Achievements",
    "Screenshots",
    "Movies",
]
df = df.drop(columns=columns_to_drop)
df.head(5)

In [ ]:
# Filter out rows that contain any word in the specified columns
df_filtered = df[~df.apply(contains_inappropriate_word, axis=1)]

# Display the filtered dataset
df_filtered

In [ ]:
# Count occurrences for each word removed
word_counts = {word: 0 for word in inappropriate_words}

for word in inappropriate_words:
    pattern = re.compile(r"\b" + re.escape(word) + r"\b", re.IGNORECASE)
    word_counts[word] = df.apply(
        lambda row: sum(
            pattern.search(str(row[col])) is not None for col in columns_to_check
        ),
        axis=1,
    ).sum()

# Plotting the results
plt.figure(figsize=(10, 6))
bars = plt.bar(word_counts.keys(), word_counts.values(), color="skyblue")

# Add labels on top of the bars
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{int(height)}",
        ha="center",
        va="bottom",
    )

plt.xlabel("Words Removed")
plt.ylabel("Number of Rows Containing the Word")
plt.title("Number of Rows Removed Containing Each Word")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df_filtered["release_year"] = df_filtered["Release date"].str.split(",").str[1]
df_filtered

In [ ]:
# df_filtered["release_year"] = pd.to_numeric(
#     df_filtered["release_year"], errors="coerce"
# )
# games_after_2020 = df_filtered[df_filtered["release_year"] > 2020]
# num_games_after_2020 = len(games_after_2020)
# print(f"Number of games released after 2015: {num_games_after_2020}")

In [ ]:
df = df_filtered
df.columns

In [ ]:
# Create the histogram plot for Price
ax = df["Price"].plot(kind="hist", bins=20, title="Price Distribution")

# Annotate the bars with the respective count values
for p in ax.patches:
    ax.annotate(
        str(int(p.get_height())),
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
    )

# Show the plot
plt.show()

In [ ]:
# Filter the data to zoom in on Price <= 200
df_filtered = df[df["Price"] <= 200]

# Replot the scatter plot for Price vs Estimated Owners (Price <= 200)
df_filtered.plot(
    kind="scatter",
    x="Price",
    y="Estimated owners",
    title="Price vs Estimated Owners (Price <= 200)",
)
plt.show()

In [ ]:
# Summary statistics for Price
price_summary = df["Price"].describe()
print(price_summary)

In [ ]:
# Create bins for Price ranges
df["Price Range"] = pd.cut(
    df["Price"],
    bins=[0, 50, 100, 150, 200, 500, 1000],
    labels=["0-50", "51-100", "101-150", "151-200", "201-500", "501+"],
)

# Count the number of games in each price range
price_range_counts = df["Price Range"].value_counts().sort_index()
print(price_range_counts)

# Plot the distribution of games across price ranges
ax = sns.barplot(x=price_range_counts.index, y=price_range_counts.values)

# Add annotations to each bar
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="bottom",
    )

plt.title("Number of Games in Each Price Range")
plt.xlabel("Price Range")
plt.ylabel("Number of Games")
plt.show()

In [ ]:
# Create the bar plot for the distribution of games by Required age
ax = (
    df["Required age"]
    .value_counts()
    .plot(kind="bar", title="Game Count by Required Age")
)

# Annotate the bars with the respective count values
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

# Show the plot
plt.show()

In [ ]:
# Bar chart for positive and negative reviews
df[["Positive", "Negative"]].sum().plot(
    kind="bar", title="Total Positive vs Negative Reviews"
)
plt.show()

In [ ]:
# Scatter plot for Price vs. Average playtime forever
df.plot(
    kind="scatter",
    x="Price",
    y="Average playtime forever",
    title="Price vs. Average Playtime Forever",
)
plt.show()

In [ ]:
# Filter to zoom in on Price range between 0 and 200
df_filtered = df[df["Price"] <= 200]

# Plot the filtered data
df_filtered.plot(
    kind="scatter",
    x="Price",
    y="Average playtime forever",
    title="Price vs Average Playtime Forever (0-200)",
)
plt.show()

In [ ]:
# Plot the number of games released per year
df["release_year"].value_counts().sort_index().plot(
    kind="bar", title="Games Released per Year"
)
plt.show()

In [ ]:
# Calculate the number of missing values in each column
missing_values = df.isnull().sum()

# Display the missing values count
print(missing_values)

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt


# # Create a heatmap to visualize missing values
# plt.figure(figsize=(10, 6))
# sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
# plt.title('Missing Values Heatmap')
# plt.show()

In [ ]:
# Calculate the number of missing values in each column
missing_values = df.isnull().sum()

# Filter columns with missing values
missing_values = missing_values[missing_values > 0]

# Plot the missing values
ax = missing_values.plot(
    kind="bar", title="Missing Values per Column", ylabel="Count of Missing Values"
)
plt.xticks(rotation=45, ha="right")

# Annotate each bar with the number of missing values
for i, v in enumerate(missing_values):
    ax.text(i, v + 0.5, str(v), ha="center", va="bottom")

plt.show()

In [ ]:
# Summarize the number of games that support each platform (True values indicate support)
platform_support = df[["Windows", "Mac", "Linux"]].apply(
    lambda x: x.value_counts().get(True, 0)
)

# Create a bar plot with the number annotated on top of each bar
ax = platform_support.plot(
    kind="bar",
    title="Comparison of Game Support for Windows, Mac, and Linux",
    ylabel="Number of Games Supported",
)
plt.xticks(rotation=0)

# Annotate the bars with the values
for i, v in enumerate(platform_support):
    ax.text(i, v + 0.1, str(v), ha="center", va="bottom")

plt.show()

In [ ]:
# Count the number of `False` values in each platform column (Windows, Mac, Linux)
platform_no_support = df[["Windows", "Mac", "Linux"]].apply(
    lambda x: (x == False).sum()
)

# Create a bar plot with the number of games that do not support each platform
ax = platform_no_support.plot(
    kind="bar",
    title="Games Without Support for Windows, Mac, and Linux",
    ylabel="Number of Games Not Supported",
)
plt.xticks(rotation=0)

# Annotate the bars with the count of `False` values
for i, v in enumerate(platform_no_support):
    ax.text(i, v + 0.1, str(v), ha="center", va="bottom")

plt.show()

In [ ]:
# Zoom in on Price range between 0 and 200
df_price_filtered = df[df["Price"] <= 200]
df_price_filtered.plot(
    kind="scatter", x="Price", y="User score", title="User Score vs Price (0-200)"
)
plt.show()

In [ ]:
# Correlation heatmap
corr = df[
    [
        "Price",
        "User score",
        "Metacritic score",
        # "Estimated owners",
        # "Reviews",
        "Positive",
        "Negative",
        "Average playtime forever",
    ]
].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap of Key Variables")
plt.show()

In [ ]:
# import seaborn as sns

# # Pair plot for key variables related to clustering
# sns.pairplot(df[['User score', 'Price', 'Estimated owners', 'Average playtime forever']])
# plt.title('Pair Plot of Key Features')
# plt.show()

In [ ]:
# Assuming 'Supported languages' is a column with comma-separated language values
df["Supported languages"] = df["Supported languages"].fillna("")

# Split the languages into separate rows (explode)
df_languages = df.assign(
    Supported_languages=df["Supported languages"].str.split(",")
).explode("Supported_languages")

# Strip any leading/trailing spaces
df_languages["Supported_languages"] = df_languages["Supported_languages"].str.strip()

# Get the count of each language
language_counts = df_languages["Supported_languages"].value_counts()
print(language_counts)

In [ ]:
# Plot the top 10 most common languages
top_languages = language_counts.head(10)
sns.barplot(x=top_languages.values, y=top_languages.index)
plt.title("Top 10 Most Common Supported Languages")
plt.xlabel("Number of Games")
plt.ylabel("Language")
plt.show()
###############################################################################################

In [ ]:
# Assuming 'Genres' is a column with comma-separated values
df["Genres"] = df["Genres"].fillna("")

# Split the genres into separate rows (explode)
df_genres = df.assign(Genres=df["Genres"].str.split(",")).explode("Genres")

# Strip any leading/trailing spaces
df_genres["Genres"] = df_genres["Genres"].str.strip()

# Get the count of each genre
genre_counts = df_genres["Genres"].value_counts()

print(genre_counts)

# Plot the top 10 most common genres
top_genres = genre_counts.head(10)
sns.barplot(x=top_genres.values, y=top_genres.index)
plt.title("Top 10 Most Common Genres")
plt.xlabel("Number of Games")
plt.ylabel("Genres")
plt.show()

In [ ]:
# Group by Genres and calculate the average price for each genre
avg_price_per_genre = (
    df_genres.groupby("Genres")["Price"].mean().sort_values(ascending=False)
)

# Plot the top 10 genres with the highest average price
sns.barplot(x=avg_price_per_genre.head(20).values, y=avg_price_per_genre.head(20).index)
plt.title("Average Price per Genre")
plt.xlabel("Average Price")
plt.ylabel("Genres")
plt.show()

In [ ]:
# Create bins for Price ranges
df_genres["Price Range"] = pd.cut(
    df_genres["Price"],
    bins=[0, 50, 100, 150, 200, 500, 1000],
    labels=["0-50", "51-100", "101-150", "151-200", "201-500", "501+"],
)

# Count how many games fall within each Price Range for each Genre
price_range_genre_count = (
    df_genres.groupby(["Price Range", "Genres"]).size().unstack().fillna(0)
)

# Plot the count of games for each Genre across Price Ranges
price_range_genre_count.plot(kind="bar", stacked=True, figsize=(10, 6))
plt.title("Price Range Distribution Across Genres")
plt.xlabel("Price Range")
plt.ylabel("Number of Games")
plt.legend(title="Genres", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()

In [ ]:
# Pair plot to explore relationships between Price and other variables
sns.scatterplot(data=df, x="Price", y="Metacritic score")
# plt.title("Pair Plot of Price and Related Variables")
plt.show()

In [ ]:
# Pair plot to explore relationships between Price and other variables
sns.scatterplot(data=df_price_filtered, x="Price", y="Metacritic score")
# plt.title("Pair Plot of Price and Related Variables")
plt.show()

In [72]:
categories = []
for poop in df["Genres"].unique().tolist():
    categories.extend(poop.split(","))
set(categories)
df.loc[df["Genres"].str.contains("Utilities")]

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Reviews,...,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,release_year,Price Range,Log User score,Score Category
31,Home Office Tasker,"Sep 8, 2021",0 - 20000,0,0,0.99,0,You no longer need to go to special applicatio...,"['English', 'Russian', 'German', 'Spanish - Sp...",NaN,...,0,lonch.me,lonch.me,NaN,Utilities,"Utilities,Time Management,Time Manipulation,So...",2021,0-50,0.0,NaN
260,SolidFace Parametric CAD Modeler 2D/3D,"Dec 22, 2014",0 - 20000,0,0,0.00,0,Part module SolidFace Sketch automatically cre...,"['English', 'Portuguese - Brazil']",NaN,...,0,"SolidFace Technology Inc.,Oscar L.",SolidFace Technology Inc.,Single-player,"Animation & Modeling,Design & Illustration,Uti...","Design & Illustration,Animation & Modeling,Uti...",2014,NaN,0.0,NaN
345,Borderless Gaming,"Jul 23, 2015",200000 - 500000,570,0,3.99,1,Borderless Gaming forces borderless-fullscreen...,"['English', 'German', 'Russian', 'French', 'Si...","“Games are best when played full screen, but m...",...,2526,"AndrewMD5,Codeusa",NaN,"Single-player,Steam Achievements,Steam Trading...",Utilities,"Utilities,Software,Indie,Anime",2015,0-50,0.0,NaN
359,Gamefuel Driver Control,"Dec 10, 2015",20000 - 50000,0,0,29.99,0,The problem: You may have hardware or devices ...,['English'],NaN,...,0,Auslogics Software,Console Classics,NaN,Utilities,Utilities,2015,0-50,0.0,NaN
450,Start10,"May 11, 2017",0 - 20000,0,0,4.99,0,Customize your Start menu for easy access to U...,"['English', 'French', 'Italian', 'German', 'Sp...",NaN,...,0,Stardock,Stardock,NaN,Utilities,"Utilities,Software",2017,0-50,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84518,Movavi Video Suite 2024 Steam Edition,"Dec 18, 2023",0 - 20000,2,0,89.99,22,Make your every video project a breeze: be it ...,"['English', 'French', 'Italian', 'German', 'Sp...",NaN,...,0,Movavi,Movavi,NaN,"Animation & Modeling,Audio Production,Design &...","Utilities,Video Production,Photo Editing,Anima...",2023,51-100,0.0,NaN
84550,Sniper Simulator,"Dec 19, 2023",0 - 20000,1,0,4.99,0,Sniper Simulator is a sniping teaching app. Th...,"['English', 'Simplified Chinese']",NaN,...,0,SUN JINZHE,SUN JINZHE,NaN,"Education,Utilities","Education,Utilities,Sniper",2023,0-50,0.0,NaN
84638,YoloX - Crosshair Overlay,"Dec 21, 2023",0 - 20000,5,0,5.99,0,Use YoloX to change your game crosshair. Inclu...,"['English', 'Simplified Chinese']",NaN,...,0,Dragonrise Games,Dragonrise Games,"Steam Workshop,Includes level editor","Action,Animation & Modeling,Design & Illustrat...","Utilities,Software Training,Action,Design & Il...",2023,0-50,0.0,NaN
84674,Timeline 存档守护者 -Manage your game save,"Dec 28, 2023",0 - 20000,0,0,3.99,0,Timeline is a game utility tool designed prima...,"['Simplified Chinese', 'English']",NaN,...,0,a little dream,a little dream,NaN,"Indie,Education,Photo Editing,Software Trainin...","Indie,Utilities,Photo Editing,Education,Softwa...",2023,0-50,0.0,NaN


In [ ]:
df = add_release_season_column(df)
df = convert_estimated_owners_to_midpoints(df)
df = add_review_columns(df)
df = add_online_offline_column(df)
df = add_player_type_numeric_column(df)